In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# Model predictions
predictions_IRV2      = np.load("predictions_IRV2.npy")
predictions_Densenet  = np.load("predictions_Densenet.npy")
predictions_Resnet50  = np.load("predictions_Resnet50.npy")
predictions_Resnet34  = np.load("predictions_Resnet34.npy")
predictions_VGG16     = np.load("predictions_VGG16.npy")

y_true = np.load("y_true.npy")

targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

all_preds = [predictions_IRV2, predictions_Densenet, predictions_Resnet50,
             predictions_Resnet34, predictions_VGG16]

N, C = predictions_IRV2.shape


In [ ]:
def normalize_mass(m):
    s = np.sum(m)
    return m / s if s != 0 else m

def ds_combine_two(m1, m2):
    C = m1.shape[0]
    m_comb = np.zeros(C)
    conflict = 0.0

    for i in range(C):
        for j in range(C):
            if i == j:
                m_comb[i] += m1[i] * m2[j]
            else:
                conflict += m1[i] * m2[j]

    if conflict < 1.0:
        m_comb = m_comb / (1.0 - conflict)
    else:
        m_comb = (m1 + m2) / 2.0

    return normalize_mass(m_comb)

def ds_fusion(predictions_list):
    fused = predictions_list[0].copy()
    for preds in predictions_list[1:]:
        for i in range(fused.shape[0]):
            fused[i] = ds_combine_two(fused[i], preds[i])
    return fused


In [ ]:
teacher_ds = ds_fusion(all_preds)


In [ ]:
all_preds_array = np.stack(all_preds, axis=0)  
disagreement = np.std(all_preds_array, axis=0).mean(axis=1)  

threshold = disagreement.mean() + disagreement.std()
print(f"Disagreement threshold: {threshold:.4f}")


In [ ]:
def bayesian_update(prior, predictions_list, eps=1e-8):
   
    log_post = np.log(prior + eps)
    for pred in predictions_list:
        log_post += np.log(pred + eps)
    posterior = np.exp(log_post)
    posterior /= posterior.sum()
    return posterior


In [ ]:
teacher_hybrid = np.zeros_like(teacher_ds)

for i in range(N):
    if disagreement[i] <= threshold:
        # Low disagreement → keep DS output
        teacher_hybrid[i] = teacher_ds[i]
    else:
        # High disagreement → use DS as prior + Bayesian update
        sample_preds = [pred[i] for pred in all_preds]
        teacher_hybrid[i] = bayesian_update(teacher_ds[i], sample_preds)


In [ ]:
np.save("teacher_hybrid_probs.npy", teacher_hybrid)


In [ ]:
y_pred_hybrid = np.argmax(teacher_hybrid, axis=1)

print(classification_report(y_true, y_pred_hybrid, target_names=targetnames))


cm = confusion_matrix(y_true, y_pred_hybrid)

plt.figure(figsize=(8,6))
plt.imshow(cm)
plt.title("Hybrid DS + Bayesian Teacher – Confusion Matrix")
plt.colorbar()
plt.xticks(range(7), targetnames, rotation=45)
plt.yticks(range(7), targetnames)

thresh_cm = cm.max() / 2
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j],
                 ha="center", va="center",
                 color="white" if cm[i, j] > thresh_cm else "black")

plt.tight_layout()
plt.ylabel("True label")
plt.xlabel("Predicted label")
plt.show()


XAI

In [ ]:
import innvestigate
import innvestigate.utils as iutils


In [ ]:
def compute_lrp(model, data_generator, method="lrp.epsilon"):

    analyzer = innvestigate.create_analyzer(method, model)

    all_relevances = []
    steps = len(data_generator)
    
    for i in range(steps):
        x_batch, y_batch = data_generator[i]
        relevances = analyzer.analyze(x_batch)
        all_relevances.append(relevances)
    
    all_relevances = np.concatenate(all_relevances, axis=0)
    return all_relevances


In [ ]:

from tensorflow.keras.models import load_model

models_and_names = {
    "IRV2": "/code/MyCode/AUG/Aug-Att/IRV2+SA.hdf5",
    "Densenet201": "/code/MyCode/AUG/Aug-Att/Densenet201+SA.hdf5",
    "ResNet50": "/code/MyCode/AUG/Aug-Att/ResNet50+SA.hdf5",
    "ResNet34": "/code/MyCode/AUG/Aug-Att/ResNet34+SA.hdf5",
    "VGG16": "/code/MyCode/AUG/Aug-Att/vgg16+SA.hdf5"
}

for model_name, model_path in models_and_names.items():
    print(f"Processing LRP for {model_name}...")
    model = load_model(model_path, compile=False)
    
    relevances = compute_lrp(model, test_batches, method="lrp.epsilon")
    
    np.save(f"LRP_{model_name}.npy", relevances)
    print(f"Saved LRP_{model_name}.npy with shape {relevances.shape}")


In [ ]:
import numpy as np

lrp_IRV2      = np.load("LRP_IRV2.npy")
lrp_Densenet  = np.load("LRP_Densenet201.npy")
lrp_ResNet50  = np.load("LRP_ResNet50.npy")
lrp_ResNet34  = np.load("LRP_ResNet34.npy")
lrp_VGG16     = np.load("LRP_VGG16.npy")

all_lrp = [lrp_IRV2, lrp_Densenet, lrp_ResNet50, lrp_ResNet34, lrp_VGG16]

print("LRP shapes:", [l.shape for l in all_lrp])
N, H, W, C = lrp_IRV2.shape


In [ ]:
def normalize_mass_map(m):
    s = np.sum(m)
    return m / s if s != 0 else m

def ds_combine_two_map(m1, m2):
    """
    Apply DS fusion on 3D or 4D importance maps (H,W,C)
    """
    m_comb = np.zeros_like(m1)
    conflict = 0.0
    
    flat1 = m1.flatten()
    flat2 = m2.flatten()
    
    for i in range(len(flat1)):
        for j in range(len(flat2)):
            if i == j:
                m_comb.flat[i] += flat1[i] * flat2[j]
            else:
                conflict += flat1[i] * flat2[j]

    if conflict < 1.0:
        m_comb = m_comb / (1.0 - conflict)
    else:
        m_comb = (m1 + m2) / 2.0
    
    return normalize_mass_map(m_comb)

def ds_fusion_map(lrp_list):
    fused = lrp_list[0].copy()
    for lrp in lrp_list[1:]:
        for i in range(fused.shape[0]):
            fused[i] = ds_combine_two_map(fused[i], lrp[i])
    return fused


In [ ]:
lrp_ds = ds_fusion_map(all_lrp)
print("LRP DS fusion done, shape:", lrp_ds.shape)


In [ ]:
all_lrp_array = np.stack(all_lrp, axis=0)  # shape: (num_models, N, H, W, C)
disagreement = np.std(all_lrp_array, axis=0).mean(axis=(1,2,3))  # mean over H,W,C

threshold = disagreement.mean() + disagreement.std()
print(f"LRP disagreement threshold: {threshold:.4f}")


In [ ]:
def bayesian_update_map(prior_map, maps_list, eps=1e-8):
    log_post = np.log(prior_map + eps)
    for m in maps_list:
        log_post += np.log(m + eps)
    posterior = np.exp(log_post)
    posterior /= posterior.sum()
    return posterior


In [ ]:
lrp_hybrid = np.zeros_like(lrp_ds)

for i in range(N):
    if disagreement[i] <= threshold:
        # low disagreement → keep DS map
        lrp_hybrid[i] = lrp_ds[i]
    else:
        # high disagreement → use DS as prior + Bayesian update
        sample_maps = [lrp[i] for lrp in all_lrp]
        lrp_hybrid[i] = bayesian_update_map(lrp_ds[i], sample_maps)


In [ ]:
np.save("LRP_hybrid.npy", lrp_hybrid)
print("Saved LRP_hybrid.npy with shape:", lrp_hybrid.shape)


In [ ]:
test_path = '/code/MyCode/AUG/HAM10000/test_dir'

test_batches = datagen.flow_from_directory(
    directory=test_path,
    target_size=input_shape[:2],
    batch_size=batch_size,
    shuffle=False
)

steps_test = len(test_batches)


In [ ]:
predictions_teacher = model.predict(test_batches, steps=steps_test, verbose=1)
np.save("teacher_predictions.npy", predictions_teacher)


In [ ]:
teacher_lrp_test = []
test_batches.reset()

for i in range(steps_test):
    X_batch, _ = test_batches.next()
    lrp_batch = compute_lrp(model, X_batch)  # LRP-0 for teacher
    teacher_lrp_test.append(lrp_batch)

teacher_lrp_test = np.vstack(teacher_lrp_test)
np.save("teacher_LRP.npy", teacher_lrp_test)


In [ ]:
import matplotlib.pyplot as plt

X_batch, _ = test_batches.next()
idx = 0

img = X_batch[idx]
lrp = teacher_lrp_test[idx].squeeze()

fig, axes = plt.subplots(1,2,figsize=(8,4))
axes[0].imshow((img - img.min()) / (img.max() - img.min()))
axes[0].set_title("Original Image")
axes[0].axis('off')

axes[1].imshow(lrp, cmap='hot')
axes[1].set_title("Teacher LRP")
axes[1].axis('off')

plt.show()
